In [1]:
from decouple import AutoConfig

In [2]:
config = AutoConfig(search_path="..")

In [3]:
assert config("ALPHA_VANTAGE_API_KEY", default=None, cast = str) is not None

In [4]:
ALPHA_VANTAGE_API_KEY = config("ALPHA_VANTAGE_API_KEY", default=None, cast = str)

In [5]:
# ALPHA_VANTAGE_API_KEY

In [9]:
import requests

api_key = ALPHA_VANTAGE_API_KEY

params = {
    "api_key" : ALPHA_VANTAGE_API_KEY,
    "ticker" : "AAPL",
    "function" : "TIME_SERIES_INTRADAY"
}

url = 'https://www.alphavantage.co/query?function={function}&symbol={ticker}&interval=5min&apikey={api_key}'.format(
    **params
)
r = requests.get(url)
data = r.json()


In [31]:
# data

In [11]:
data.keys()

dict_keys(['Meta Data', 'Time Series (5min)'])

In [13]:
dataset_key =[ x for x in list(data.keys()) if not x.lower() == "meta data"][0]
dataset_key

'Time Series (5min)'

In [14]:
results = data[dataset_key]

In [34]:
import pytz
from datetime import datetime

In [20]:
# for timestamp_str in results.keys():
#     print(timestamp_str, results.get(timestamp_str))
    

In [22]:
timestamp_str = list(results.keys())[0]
timestamp_str

'2025-07-30 19:55:00'

In [36]:
timestamp_format = '%Y-%m-%d %H:%M:%S' # dateutils
eastern = pytz.timezone('US/Eastern')
utc = pytz.utc
timestamp = eastern.localize(datetime.strptime(timestamp_str, timestamp_format)).astimezone(utc)
timestamp

datetime.datetime(2025, 7, 30, 23, 55, tzinfo=<UTC>)

In [25]:
result_val = results.get(timestamp_str)

In [39]:
import pytz
from datetime import datetime
from decimal import Decimal

def transform_alphavantage_result(timestamp_str, result):
    # unix_timestamp = result.get('t') / 1000.0
    # utc_timestamp = datetime.fromtimestamp(unix_timestamp, tz = pytz.timezone('UTC'))
    timestamp_format = '%Y-%m-%d %H:%M:%S' # dateutils
    eastern = pytz.timezone('US/Eastern')
    utc = pytz.utc
    utc_timestamp = eastern.localize(datetime.strptime(timestamp_str, timestamp_format)).astimezone(utc)
    return {
        'open_price' : Decimal(result['1. open']),
        'close_price' : Decimal(result['4. close']),
        'high_price' : Decimal(result['2. high']),
        'low_price' : Decimal(result['3. low']),
        # 'number_of_trades' : None,
        'volume' : int(result['5. volume']),
        # 'volume_weighted_average' : None,
        'time' : utc_timestamp
    }

In [40]:
transform_alphavantage_result(timestamp_str, result_val)

{'open_price': Decimal('208.9200'),
 'close_price': Decimal('208.7300'),
 'high_price': Decimal('208.9200'),
 'low_price': Decimal('208.6573'),
 'volume': 28880,
 'time': datetime.datetime(2025, 7, 30, 23, 55, tzinfo=<UTC>)}

In [41]:
for timestamp_str in results.keys():
    new_data = transform_alphavantage_result(timestamp_str, results.get(timestamp_str))
    print(new_data)

{'open_price': Decimal('208.9200'), 'close_price': Decimal('208.7300'), 'high_price': Decimal('208.9200'), 'low_price': Decimal('208.6573'), 'volume': 28880, 'time': datetime.datetime(2025, 7, 30, 23, 55, tzinfo=<UTC>)}
{'open_price': Decimal('208.8401'), 'close_price': Decimal('208.9187'), 'high_price': Decimal('208.9900'), 'low_price': Decimal('208.7834'), 'volume': 11265, 'time': datetime.datetime(2025, 7, 30, 23, 50, tzinfo=<UTC>)}
{'open_price': Decimal('208.7300'), 'close_price': Decimal('208.9100'), 'high_price': Decimal('208.9900'), 'low_price': Decimal('208.7300'), 'volume': 19483, 'time': datetime.datetime(2025, 7, 30, 23, 45, tzinfo=<UTC>)}
{'open_price': Decimal('208.6400'), 'close_price': Decimal('208.7400'), 'high_price': Decimal('208.7400'), 'low_price': Decimal('208.5700'), 'volume': 10717, 'time': datetime.datetime(2025, 7, 30, 23, 40, tzinfo=<UTC>)}
{'open_price': Decimal('208.5500'), 'close_price': Decimal('208.6500'), 'high_price': Decimal('208.7000'), 'low_price': 